# Carbon Aware Federated Learning using Flower and LowCarb, powered by the Carbon-Aware-SDK
This notebook follows along the official "Introduction to Federated Learning" from the Flower [tutorial](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html) using PyTorch and the CIFAR10 dataset.

The **two** extra steps to make your Flower application carbon aware are highlighted along the way.

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

from random import sample
import flwr as fl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

DEVICE = torch.device("cpu")
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")





In [ ]:
NUM_CLIENTS = 20
BATCH_SIZE = 32
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(num_clients=NUM_CLIENTS)

## Implementation of the Neural Net
...using the CIFAR10 dataset. For an in-depth explanation see the [Flower tutorial](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html).

In [ ]:
def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

def weighted_average(metrics):
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

## Implementation of the Flower Client
The `FlowerClient` for this demo is based on the [Flower tutorial](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html).

Besides this, granting your client Carbon-Awareness is as non-invasive as it gets.

To make your client implementation compatible with `lowcarb`'s `LowcarbClientManager` (which we'll meet further down the line), you have to make sure only one thing:
its `get_properties()` must return a `dict` that contains the client's `location` as a `str`, e.g. *'uksouth'*, *'westeurope'*, *'westus'*

```
class Carbon_Aware_Client():
    def __init__(self, location: str):
        self.location = location

    # Make sure the get_properties() method returns the client's location inside a dict
    def get_properties(self, config) -> dict:
        return {'location': self.location}
```


That's it.

**Note: If the client's location changes throughout its lifetime, just update it dynamically in `get_properties()`. The `LowcarbClientManager` queries it each round.**

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, location: str, *args, **kwargs):
        super(FlowerClient, self).__init__(*args, **kwargs)
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

        ### Giving the client a location upon instantiation
        self.location = location

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

    # Make sure the get_properties() method returns the client's location inside a dict
    def get_properties(self, config) -> dict:
        return {'location': self.location}

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

# Flower Simulation Setup
## Generating Clients at Locations around the World

For performance reasons, the clients are not persistent and are instantiated (`NUM_CLIENTS` in total) each round by the simulation calling `client_generator(cid: str)` with `['01', '02', '03', ...]` (see the [Flower tutorial](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html)).
Thus, we sample the clients-location-mapping `client_regions_by_cid` from a list of available regions `available_regions`, which are supported by the `Carbon-Aware-SDK`.
Important: the mapping has to be done outside the scope of `client_generator`.

IMPORTANT: this is not the carbon-aware sampling for each federated learning round. The `client_generator` only creates the pool of available clients (from which the `LowcarbClientManager` will sample clients in a carbon-aware manner later).

In [ ]:
available_regions = ['westcentralus', 'ukwest', 'uksouth', 'westeurope', 'westus', 'australiacentral', 'australiaeast', 'swedencentral', 'norwaywest', 'norwayeast', 'northeurope', 'centralus', 'francesouth', 'francecentral']
client_regions_by_cid = [sample(available_regions, 1)[0] for i in range(0, NUM_CLIENTS)]

def client_generator(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader, location=client_regions_by_cid[int(cid)])

## Selection the Federated Learning Strategy
as in the [Flower tutorial](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html).
When it comes to the federated learning strategy no modifications are required. The carbon-aware magic happens in the `LowcarbClientManager` up next.

In [ ]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Sample 2.5% of available clients for training
    fraction_evaluate=0.025,  # Sample 2.5% of available clients for evaluation
    min_fit_clients=10,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=10,  # Wait until all 10 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)

## Using the LowcarbClientManager
For your Flower application to become carbon-aware and automatically select only clients with the least possible carbon footprint, use the `LowcarbClientManager`.
`LowcarbClientManager` is powered by the Carbon-Aware-SDK's WebApi. Feel free to host it locally or use a remote endpoint.

In [ ]:
from lowcarb.lowcarb_client_manager import LowcarbClientManager

carbon_aware_sdk_api = 'https://carbon-aware-api.azurewebsites.net' # URL to the Carbon-Aware_SDK's WebApi
workload_duration = 15  # estimated workload of each training in minutes
forecast_window = 12    # forecast window considered by the client manager in hours
lowcarb_client_manager = LowcarbClientManager(api_host=carbon_aware_sdk_api, workload_duration=15, forecast_window=12)

## Starting the Simulation

In [ ]:
# Start simulation
fl.simulation.start_simulation(
    client_fn=client_generator,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_manager=lowcarb_client_manager,  # pass the lowcarb_client_manager to the simulation
)